In [1]:
import os
import cv2
import dlib
import numpy as np

In [2]:
def preprocess_face(face):
    # Example: Convert to grayscale
    gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

    # Example: Resize to a standard size (adjust as needed)
    resized_face = cv2.resize(gray_face, (224, 224))

    return resized_face

In [3]:
def extract_frames_and_faces(video_path, output_dir,reduced_output_dir, video_count, scale_factor=1.5, every_nth_frame=25):

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    face_detector = dlib.get_frontal_face_detector()
    frame_count = 0
    num_faces = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray_frame)

        # Handle no faces detected
        if not faces:
            continue

        num_faces = len(faces)  # Update face count after filtering

        # Create separate folder for each face if multiple faces detected
        if num_faces > 1:
            face_id = 0
            for face in faces:
                x, y, w, h = face.left(), face.top(), face.width(), face.height()

                # Increase the size of the rectangle by a scale factor
                x -= int(w * (scale_factor - 1) / 2)
                y -= int(h * (scale_factor - 1) / 2)
                w = int(w * scale_factor)
                h = int(h * scale_factor)

                # Ensure the new rectangle coordinates are within the frame boundaries
                x = max(0, x)
                y = max(0, y)
                w = min(frame.shape[1] - x, w)
                h = min(frame.shape[0] - y, h)

                cropped_face = frame[y:y + h, x:x + w]
                processed_face = preprocess_face(cropped_face)

                face_output_dir = os.path.join(output_dir, f"face_{face_id}")
                if not os.path.exists(face_output_dir):
                    os.makedirs(face_output_dir)

                save_path = os.path.join(face_output_dir, f"frame_{frame_count}_{video_count}.jpg")
                cv2.imwrite(save_path, processed_face)

                face_id += 1

        # Single face detected: Save all frames to main folder, and every nth frame to reduced folder
        else:
            face = faces[0]
            x, y, w, h = face.left(), face.top(), face.width(), face.height()

            # Increase the size of the rectangle by a scale factor
            x -= int(w * (scale_factor - 1) / 2)
            y -= int(h * (scale_factor - 1) / 2)
            w = int(w * scale_factor)
            h = int(h * scale_factor)

            # Ensure the new rectangle coordinates are within the frame boundaries
            x = max(0, x)
            y = max(0, y)
            w = min(frame.shape[1] - x, w)
            h = min(frame.shape[0] - y, h)

            cropped_face = frame[y:y + h, x:x + w]
            processed_face = preprocess_face(cropped_face)

            save_path = os.path.join(output_dir, f"frame_{frame_count}_{video_count}.jpg")
            cv2.imwrite(save_path, processed_face)

            # Extract every nth frame for reduced dataset
            if frame_count % every_nth_frame == 0:
                if not os.path.exists(reduced_output_dir):
                    os.makedirs(reduced_output_dir)

                reduced_save_path = os.path.join(reduced_output_dir, f"frame_{frame_count}_{video_count}.jpg")
                cv2.imwrite(reduced_save_path, processed_face)
        frame_count += 1
    cap.release()



In [4]:
def is_video(filepath):
  video_extensions = (".mp4", ".avi", ".mkv", ".wmv", ".flv", ".mov")
  _, extension = os.path.splitext(filepath)
  return extension.lower() in video_extensions


In [5]:
if __name__ == "__main__":
    video_folder = "../Forensic++ Dataset/FaceForensic/manipulated_sequences/DeepFakeDetection"  # Replace with your video folder path
    main_output_folder = "Dataset/Fake"  # Replace with your output folder path
    reduced_output_dir = "Reduced_Dataset/Fake"
    found = 0
    video_count = 347
    for video_filename in os.listdir(video_folder):
        if video_filename == "03_14__podium_speech_happy__7JPPCV50.mp4":
            found = 1
            print("found")
        if found == 1 and is_video(video_filename):
            print(video_filename)
            output_folder = os.path.join(main_output_folder, video_filename)
            reduced_output = os.path.join(reduced_output_dir, video_filename)
            video_path = os.path.join(video_folder, video_filename)
            extract_frames_and_faces(video_path, output_folder, reduced_output, video_count)
            print(video_filename)


found
03_14__podium_speech_happy__7JPPCV50.mp4
03_14__podium_speech_happy__7JPPCV50.mp4
03_14__podium_speech_happy__Q9NSXM88.mp4
03_14__podium_speech_happy__Q9NSXM88.mp4
03_14__talking_against_wall__ZC2KYASW.mp4
03_14__talking_against_wall__ZC2KYASW.mp4
03_14__walking_down_indoor_hall_disgust__KJ221YN0.mp4
03_14__walking_down_indoor_hall_disgust__KJ221YN0.mp4
03_14__walking_down_street_outside_angry__H0VQHGS3.mp4
03_14__walking_down_street_outside_angry__H0VQHGS3.mp4
03_14__walk_down_hall_angry__P1L5PF4I.mp4
03_14__walk_down_hall_angry__P1L5PF4I.mp4
03_15__exit_phone_room__GZNLO4X9.mp4
03_15__exit_phone_room__GZNLO4X9.mp4
03_15__kitchen_pan__AIOM1U5V.mp4
03_15__kitchen_pan__AIOM1U5V.mp4
03_15__kitchen_pan__DNUJD8M2.mp4
03_15__kitchen_pan__DNUJD8M2.mp4
03_15__kitchen_still__AIOM1U5V.mp4
03_15__kitchen_still__AIOM1U5V.mp4
03_15__meeting_serious__DNUJD8M2.mp4
03_15__meeting_serious__DNUJD8M2.mp4
03_15__meeting_serious__OY4I7ZU8.mp4
03_15__meeting_serious__OY4I7ZU8.mp4
03_15__meeting_serio

KeyboardInterrupt: 